# 02 — Photometry Basics (Aperture Photometry)

## What you’ll learn
- Instrumental flux vs. magnitude
- Background estimation (local annulus)
- Aperture radius trade-offs (SNR vs contamination)
- Differential photometry (cancel systematics)

We’ll reuse a synthetic star field from scratch (so it runs anywhere).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter

np.random.seed(1)

def imshow(img, title=None, vmin=None, vmax=None):
    plt.figure(figsize=(6,5))
    plt.imshow(img, origin="lower", vmin=vmin, vmax=vmax)
    plt.colorbar(label="counts")
    if title: plt.title(title)
    plt.xlabel("x [pix]")
    plt.ylabel("y [pix]")
    plt.tight_layout()
    plt.show()


## 1) Create a synthetic image + noise

We’ll generate:
- several stars (Gaussian PSFs)
- sky background
- noise (shot + read)


In [ ]:
def gaussian_psf(ny, nx, x0, y0, flux, fwhm):
    y, x = np.mgrid[0:ny, 0:nx]
    sigma = fwhm / 2.355
    return flux * np.exp(-((x-x0)**2 + (y-y0)**2)/(2*sigma**2))

def make_scene(ny=150, nx=150, stars=None, sky=80.0, fwhm=3.0):
    img = np.full((ny,nx), sky, float)
    for (x0,y0,flux) in stars:
        img += gaussian_psf(ny,nx,x0,y0,flux,fwhm)
    return img

stars = [
    (50.2, 65.4, 15000),   # target star
    (95.1, 80.8,  9000),   # comparison
    (30.8, 40.1, 12000),   # comparison
    (110.3, 35.6, 5000),   # field star
]
truth = make_scene(stars=stars, sky=80.0, fwhm=3.0)

# noise model
read_noise = 6.0
noisy = np.random.poisson(np.clip(truth,0,None)).astype(float)
noisy += np.random.normal(0, read_noise, size=noisy.shape)

imshow(noisy, "Noisy image", vmax=np.percentile(noisy, 99.5))


## 2) A simple aperture photometry function (no extra packages)

Steps:
1. select pixels inside a circular aperture → **source + sky**
2. estimate sky level from an annulus
3. subtract sky contribution from aperture sum


In [ ]:
def circular_mask(shape, x0, y0, r):
    ny, nx = shape
    y, x = np.mgrid[0:ny, 0:nx]
    return (x-x0)**2 + (y-y0)**2 <= r**2

def annulus_mask(shape, x0, y0, r_in, r_out):
    ny, nx = shape
    y, x = np.mgrid[0:ny, 0:nx]
    rr2 = (x-x0)**2 + (y-y0)**2
    return (rr2 >= r_in**2) & (rr2 <= r_out**2)

def aperture_photometry(img, x0, y0, r_ap=4.0, r_in=6.0, r_out=10.0, sky_stat="median"):
    ap = circular_mask(img.shape, x0, y0, r_ap)
    an = annulus_mask(img.shape, x0, y0, r_in, r_out)

    ap_sum = np.sum(img[ap])
    sky_vals = img[an]
    if sky_stat == "median":
        sky = np.median(sky_vals)
    else:
        sky = np.mean(sky_vals)

    n_ap = np.sum(ap)
    flux = ap_sum - sky * n_ap
    return {"flux": flux, "sky_per_pix": sky, "n_ap": n_ap}

# try it on the "target" star
x0,y0,_ = stars[0]
res = aperture_photometry(noisy, x0, y0)
res


## 3) Visualize the aperture + annulus on the image


In [ ]:
def plot_aperture(img, x0, y0, r_ap, r_in, r_out, title=None):
    plt.figure(figsize=(6,5))
    plt.imshow(img, origin="lower", vmax=np.percentile(img, 99.5))
    theta = np.linspace(0, 2*np.pi, 400)
    for r, ls in [(r_ap,'-'), (r_in,'--'), (r_out,'--')]:
        plt.plot(x0 + r*np.cos(theta), y0 + r*np.sin(theta), ls, linewidth=2)
    plt.scatter([x0],[y0], s=30)
    plt.colorbar(label="counts")
    plt.title(title or "Aperture (solid) and sky annulus (dashed)")
    plt.xlabel("x [pix]")
    plt.ylabel("y [pix]")
    plt.tight_layout()
    plt.show()

plot_aperture(noisy, x0, y0, r_ap=4, r_in=6, r_out=10)


## 4) Instrumental magnitude

Astronomy often converts fluxes to magnitudes:
\[
m_{\mathrm{inst}} = -2.5 \log_{10}(F)
\]

Absolute calibration requires a zero point, but **relative** work (differential photometry) often doesn’t.


In [ ]:
def inst_mag(flux):
    flux = np.asarray(flux)
    return -2.5*np.log10(np.clip(flux, 1e-12, None))

for i,(x0,y0,flux_true) in enumerate(stars):
    r = aperture_photometry(noisy, x0, y0)
    print(i, "flux_est=", r["flux"], "m_inst=", inst_mag(r["flux"]))


## 5) Differential photometry (target / comparison)

If transparency varies (thin clouds), both target and comparison change similarly.  
Taking a ratio helps cancel those trends.

We’ll simulate 50 exposures with a varying “throughput” and recover a stable differential light curve.


In [ ]:
def simulate_exposure(throughput=1.0, jitter=0.2):
    # jitter star centroids slightly
    jittered = [(x+np.random.normal(0,jitter),
                 y+np.random.normal(0,jitter),
                 f*throughput) for (x,y,f) in stars]
    img = make_scene(stars=jittered, sky=80, fwhm=3.0)
    img = np.random.poisson(np.clip(img,0,None)).astype(float)
    img += np.random.normal(0, read_noise, size=img.shape)
    return img, jittered

n = 50
t = np.arange(n)

# throughput varies over time (e.g., transparency)
throughput = 0.8 + 0.2*np.sin(2*np.pi*t/25) + 0.05*np.random.normal(size=n)

target_flux = []
comp_flux = []

for k in range(n):
    img, jittered = simulate_exposure(throughput=throughput[k])
    # target = star 0; comparison = average of stars 1 and 2
    x0,y0,_ = jittered[0]
    f_t = aperture_photometry(img, x0, y0, r_ap=4, r_in=6, r_out=10)["flux"]
    target_flux.append(f_t)

    f_cs = []
    for j in [1,2]:
        xj,yj,_ = jittered[j]
        f_c = aperture_photometry(img, xj, yj, r_ap=4, r_in=6, r_out=10)["flux"]
        f_cs.append(f_c)
    comp_flux.append(np.mean(f_cs))

target_flux = np.array(target_flux)
comp_flux = np.array(comp_flux)

rel = target_flux / comp_flux

plt.figure(figsize=(7,4))
plt.plot(t, target_flux/np.median(target_flux), label="Target (raw)", marker="o", linewidth=1)
plt.plot(t, comp_flux/np.median(comp_flux), label="Comparison (raw)", marker="o", linewidth=1)
plt.plot(t, rel/np.median(rel), label="Target / Comparison", marker="o", linewidth=2)
plt.xlabel("exposure #")
plt.ylabel("normalized flux")
plt.legend()
plt.tight_layout()
plt.show()


## Try it
- Change aperture radius and see how noise changes.
- Move a bright star closer to the target to see contamination.
- Replace median sky with mean sky and observe sensitivity to outliers.

Next notebook: **03 — Time Series, Periods & Transits**
